# Using OpenAI with ROS

<img src="img/robotignite_logo_text.png" width="400"/>

## Unit 5: Save and Load the trained policy

<p style="background:green;color:white;">SUMMARY</p>

Estimated time to completion: <b>1 hour</b><br><br>
In this unit, you are going to see how to save the learned policy and how to load it to apply what the agent has learned.

<p style="background:green;color:white;">END OF SUMMARY</p>

So, in the previous Units, you have been learning how to create the whole environments structure in order to get your robot training, and how this structure works. But now... what do you do when the training has finished? What do you do with all the new knowledge acquired? Well... you save it, of course!

In Artificial Intellegence, we usually reffer to all this knowledge obtained during a training session as the **learned policy**. In this Unit, you are going to learn how to save this policy, and also how to load it afterwards so that your robot can use this knowledge to decide what to do.

## Adding the save and load functions

The first thing we are going to do is to do an small modification to the **qlearn.py** script that you've using to train the robots until now. Basically, we are going to add 2 new functions to the script: 
* **save() function**: It will save the trained policy at the end of a training session to a new file.


* **load() function**: It will load the trained policy from an existing file.

So, at the end of the **qlearn.py** file, add the following 2 new functions:

In [ ]:
def save(self, filename):
    np.save(filename, self.q)

def load(self, filename):
    self.q = np.load(filename).item()

As you can see, it's quite self-explanatory. We are basically using the **numpy** save and load methods to save and load the learned policy. So, at the end, you should have a file like this one:

<p style="background:green;color:white;">**qlearn.py**</p>

In [ ]:
import random
import numpy as np

class QLearn:
    def __init__(self, actions, epsilon, alpha, gamma):
        self.q = dict()
        self.epsilon = epsilon  # exploration constant
        self.alpha = alpha      # discount constant
        self.gamma = gamma      # discount factor
        self.actions = actions

    def getQ(self, state, action):
        return self.q.get((state, action), 0.0)

    def learnQ(self, state, action, reward, value):
        '''
        Q-learning:
            Q(s, a) += alpha * (reward(s,a) + max(Q(s') - Q(s,a))            
        '''
        oldv = self.q.get((state, action), None)
        if oldv is None:
            self.q[(state, action)] = reward
        else:
            self.q[(state, action)] = oldv + self.alpha * (value - oldv)

    def chooseAction(self, state, return_q=False):
        q = [self.getQ(state, a) for a in self.actions]
        maxQ = max(q)

        if random.random() < self.epsilon:
            minQ = min(q); mag = max(abs(minQ), abs(maxQ))
            # add random values to all the actions, recalculate maxQ
            q = [q[i] + random.random() * mag - .5 * mag for i in range(len(self.actions))] 
            maxQ = max(q)

        count = q.count(maxQ)
        # In case there're several state-action max values 
        # we select a random one among them
        if count > 1:
            best = [i for i in range(len(self.actions)) if q[i] == maxQ]
            i = random.choice(best)
        else:
            i = q.index(maxQ)

        action = self.actions[i]        
        if return_q: # if they want it, give it!
            return action, q
        return action

    def learn(self, state1, action1, reward, state2):
        maxqnew = max([self.getQ(state2, a) for a in self.actions])
        self.learnQ(state1, action1, reward, reward + self.gamma*maxqnew)

    def save(self, filename):
        np.save(filename, self.q)

    def load(self, filename):
        self.q = np.load(filename).item()

<p style="background:green;color:white;">**qlearn.py**</p>

## Modify the training scripts

Great! So now that we've added these 2 new functionalities to our Qlearn algorithm, let's use them in our training scripts. First of all, let's make sure that we save the trained policy whenever a training session has finished.

For that, let's add to our **start_training.py** script the code line in order to save the learned policy. The line will be like this:

In [ ]:
qlearn.save("file_name.npy")

And where will we add it? Well, the best option is at the end of the script, right before we close the environment:

In [ ]:
qlearn.saveQ("file_name.npy")

env.close()

Excellent! So we are now saving our training before it finishes, which it is a great improvement already. Now let's see how we can do it in order to load an already trained policy, in the case we already have one. 

There are actually many ways to do this. For instance, you could create a totally new script, which just loads a trained policy and uses it in order to decide what to do. For this case, though, we are going to integrate into the same script. Have a look at the following code snippet:

In [ ]:
qfile = "qlearn_states.npy"
if (os.path.exists(qfile)):
    print("Loading from file:",qfile)
    qlearn.loadQ(qfile)

It's quite simple, actually. We define a file name for our trained policy, in this case, *qlearn_states.npy*. Then, we check if this file already exists. If it already exists, then we load it. If it doesn't exist, then it will continue as usual, and save the file at the end of the training.

So, at the end, you would have something similar to this:

<p style="background:green;color:white;">**start_training.py**</p>

In [ ]:
#!/usr/bin/env python

import gym
import time
import numpy
import random
import time
import qlearn
from gym import wrappers

# ROS packages required
import rospy
import rospkg
import os

# import our training environment
from openai_ros.task_envs.cartpole_stay_up import stay_up


if __name__ == '__main__':
    
    rospy.init_node('cartpole_gym', anonymous=True, log_level=rospy.WARN)

    # Create the Gym environment
    env = gym.make('CartPoleStayUp-v0')
    rospy.loginfo ( "Gym environment done")
        
    # Set the logging system
    rospack = rospkg.RosPack()
    pkg_path = rospack.get_path('cartpole_v0_training')
    outdir = pkg_path + '/training_results'
    env = wrappers.Monitor(env, outdir, force=True) 
    rospy.loginfo ( "Monitor Wrapper started")

    last_time_steps = numpy.ndarray(0)

    # Loads parameters from the ROS param server
    # Parameters are stored in a yaml file inside the config directory
    # They are loaded at runtime by the launch file
    Alpha = rospy.get_param("/cartpole_v0/alpha")
    Epsilon = rospy.get_param("/cartpole_v0/epsilon")
    Gamma = rospy.get_param("/cartpole_v0/gamma")
    epsilon_discount = rospy.get_param("/cartpole_v0/epsilon_discount")
    nepisodes = rospy.get_param("/cartpole_v0/nepisodes")
    nsteps = rospy.get_param("/cartpole_v0/nsteps")
    running_step = rospy.get_param("/cartpole_v0/running_step")

    # Initialises the algorithm that we are going to use for learning
    qlearn = qlearn.QLearn(actions=range(env.action_space.n),
                    alpha=Alpha, gamma=Gamma, epsilon=Epsilon)
    initial_epsilon = qlearn.epsilon

    start_time = time.time()
    highest_reward = 0
    
    # Check if it already exists a training policy, and load it if so
    qfile = "qlearn_states.npy"
    if (os.path.exists(qfile)):
        print("Loading from file:",qfile)
        qlearn.loadQ(qfile)

    # Starts the main training loop: the one about the episodes to do
    for x in range(nepisodes):
        rospy.logdebug("############### START EPISODE => " + str(x))
        
        cumulated_reward = 0  
        done = False
        if qlearn.epsilon > 0.05:
            qlearn.epsilon *= epsilon_discount
        
        # Initialize the environment and get first state of the robot
        
        observation = env.reset()
        state = ''.join(map(str, observation))
        
        # Show on screen the actual situation of the robot
        # for each episode, we test the robot for nsteps
        for i in range(nsteps):
            
            rospy.loginfo("############### Start Step => "+str(i))
            # Pick an action based on the current state
            action = qlearn.chooseAction(state)
            rospy.loginfo ("Next action is: %d", action)
            # Execute the action in the environment and get feedback
            observation, reward, done, info = env.step(action)
            rospy.loginfo(str(observation) + " " + str(reward))
            cumulated_reward += reward
            if highest_reward < cumulated_reward:
                highest_reward = cumulated_reward

            nextState = ''.join(map(str, observation))

            # Make the algorithm learn based on the results
            #rospy.logwarn("############### State we were => " + str(state))
            #rospy.logwarn("############### Action that we took => " + str(action))
            #rospy.logwarn("############### Reward that action gave => " + str(reward))
            #rospy.logwarn("############### State in which we will start next step => " + str(nextState))
            qlearn.learn(state, action, reward, nextState)

            if not(done):
                state = nextState
            else:
                rospy.loginfo ("DONE")
                last_time_steps = numpy.append(last_time_steps, [int(i + 1)])
                break
            rospy.loginfo("############### END Step =>" + str(i))
            #raw_input("Next Step...PRESS KEY")
            #rospy.sleep(2.0)
        m, s = divmod(int(time.time() - start_time), 60)
        h, m = divmod(m, 60)
        rospy.logwarn ( ("EP: "+str(x+1)+" - [alpha: "+str(round(qlearn.alpha,2))+" - gamma: "+str(round(qlearn.gamma,2))+" - epsilon: "+str(round(qlearn.epsilon,2))+"] - Reward: "+str(cumulated_reward)+"     Time: %d:%02d:%02d" % (h, m, s)))

    
    rospy.loginfo ( ("\n|"+str(nepisodes)+"|"+str(qlearn.alpha)+"|"+str(qlearn.gamma)+"|"+str(initial_epsilon)+"*"+str(epsilon_discount)+"|"+str(highest_reward)+"| PICTURE |"))

    l = last_time_steps.tolist()
    l.sort()

    #print("Parameters: a="+str)
    rospy.loginfo("Overall score: {:0.2f}".format(last_time_steps.mean()))
    rospy.loginfo("Best 100 score: {:0.2f}".format(reduce(lambda x, y: x + y, l[-100:]) / len(l[-100:])))
    
    #Save the trained policy
    qlearn.save(qfile)

    env.close()


<p style="background:green;color:white;">**start_training.py**</p>

<p style="background:#EE9023;color:white;">**Exercise 5.1**</p>
<br>

a) With your new modified files, start a new training session and save the learned policy into a file.

b) Then, load the trained policy and compare the results with the previous training session. Have they improved?

<p style="background:#EE9023;color:white;">**End of Exercise 5.1**</p>